In [ ]:
from gensim.corpora import Dictionary
import tqdm
import sys
from gensim import corpora
from gensim import corpora, models, similarities
import numpy as np
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
dct = Dictionary(prune_at=None)

In [ ]:
with open ("../data/docs.tsv") as fin:
    for i, doc in enumerate (fin):
        doc = doc.strip ().split("\t")[1:]
        for d in doc:
            dct.add_documents ([d.split (" ")], prune_at=None)
            
        if i % 10000 == 0:
            sys.stdout.write ("\r {}". format (i))

In [ ]:
dct = dct.load ("../result/dct.dump")

In [ ]:
queries = {}
with open ("../data/queries.tsv") as fin:
    for q in fin:
        q = q.strip ().decode ("utf-8").upper ().split ("\t")
        queries[int (q[0])] = dct.doc2bow (q[1].split ())

In [ ]:
def doc_reader (title):
    doc_num = 0 if title else 1
    
    with open ("../data/docs.tsv") as fin:
        for i, doc in enumerate (fin):
            doc = doc.strip ().split("\t")[1:]
            if len (doc) > doc_num:
                yield dct.doc2bow(doc[doc_num].split (" "))
            else:
                yield []

In [ ]:
corpora.MmCorpus.serialize('../result/corpus_body.mm', doc_reader (title=False))

In [ ]:
corpus_body = corpora.MmCorpus('../result/corpus_body.mm')

In [ ]:
tfidf_body = models.TfidfModel(corpus_body)

In [ ]:


index_body = similarities.SparseMatrixSimilarity(tfidf_body[corpus_body], num_features=29240842)

with open ("../result/result_2.csv", "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for key in tqdm.tqdm (sorted (queries.iterkeys ())):
        for doc in np.argsort (index_body[tfidf_body[queries_body[key]]])[-5:][::-1]:
            fout.write ("{},{}\n".format (key, doc))

In [ ]:
from gensim.corpora.textcorpus import TextCorpus
from gensim.test.utils import datapath, get_tmpfile
from gensim.similarities import Similarity

In [ ]:
index_temp = get_tmpfile("index")

In [ ]:
index = Similarity(index_temp, tfidf_body[corpus_body], num_features=29240842)

In [ ]:
with open ("../result/result_3.csv", "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for key in tqdm.tqdm (sorted (queries.iterkeys ())):
        for doc in np.argsort (index[tfidf_body[queries[key]]])[-5:][::-1]:
            fout.write ("{},{}\n".format (key, doc))